# Boosting

### Load and pre-process data

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'

%run Data\ load.ipynb

give
False     265
True     4233
Name: give, dtype: int64
0.93739664540515
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4498 entries, 0 to 4586
Data columns (total 53 columns):
tender_num                        4498 non-null int64
deal_date                         4498 non-null datetime64[ns]
report_date                       4498 non-null datetime64[ns]
period                            4498 non-null int64
credit_amount                     4498 non-null float64
return_amount                     4498 non-null float64
rate                              4498 non-null float64
outstanding                       4498 non-null float64
return_date                       4498 non-null datetime64[ns]
expiration_date                   4498 non-null datetime64[ns]
tender_credit_amount              4498 non-null float64
tender_return_amount              4498 non-null float64
tender_interest_rate              4498 non-null float64
credit_amount_average             4498 non-null float64
credit_amo

### Imports

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit, KFold, cross_val_score, cross_val_predict, train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_squared_error, r2_score, explained_variance_score, mean_squared_error, confusion_matrix
import xgboost as xgb

import numpy as np
import pylab as plt
import seaborn as sns

### Feature selection

In [5]:
features = [
    'deal_date_month',
    'expiration_date_month',
    'period',
    'credit_amount',
    'credit_amount_total',
    'creditors_count',
    'purpose_code',
    'rate',
#    'debt',
    'loan_count',
    'business_level',
    'reg_country_id',
    'reg_city_code',
    'pass_date_year',
    'empty_info',
    'return_rate',
    'return_amount_rate',
    'tender_interest_rate',
    'tender_credit_amount',
    'credit_amount_average',
    'credit_amount_to_average_ratio',
    'average_repeated_loan_count',
#    'connotation'
]
#deals[['credit_amount_total','return_rate', 'return_amount_rate']].head()

In [37]:
y = deals['give']
X = deals[features]

train_X, test_X, train_y, test_y = train_test_split(X.values, y.values, test_size=0.25)
print("Taille des predictor sur le train set : ", train_X.shape)
print("Taille de la target sur le train set : ", train_y.shape)
print("Taille des predictor sur le test set : ", test_X.shape)
print("Taille de la target sur le test set : ", test_y.shape)

counts = deals.groupby('give')['give'].count()

Taille des predictor sur le train set :  (3373, 21)
Taille de la target sur le train set :  (3373,)
Taille des predictor sur le test set :  (1125, 21)
Taille de la target sur le test set :  (1125,)


In [38]:
from xgboost import XGBRegressor
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(train_X, train_y, early_stopping_rounds=2, 
             eval_set=[(test_X, test_y)], verbose=False)

train_y_pred = my_model.predict(train_X)
auc = roc_auc_score(train_y, train_y_pred)
print("Performance sur le train : ", auc)

test_y_pred = my_model.predict(test_X)
auc = roc_auc_score(test_y, test_y_pred)
print("Performance sur le test : ", auc)

Performance sur le train :  0.8982679962198977
Performance sur le test :  0.7685646829165349


In [39]:
from xgboost import XGBClassifier
xgb1 = XGBClassifier(
    learning_rate =0.1, n_estimators=1000,
    max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
    objective= 'binary:logistic', nthread=4, scale_pos_weight=counts[0]/counts[1], seed=27)

xgb1.fit(train_X, train_y)

auc = roc_auc_score(train_y, xgb1.predict(train_X))
print("Performance sur le train : ", auc)

auc = roc_auc_score(test_y, xgb1.predict(test_X))
print("Performance sur le test : ", auc)

Performance sur le train :  0.9995283018867924
Performance sur le test :  0.6085588793922128


In [40]:
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=5,
                                                  min_child_weight=1,
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8,
                                                  objective='binary:logistic', nthread=4, scale_pos_weight=counts[0]/counts[1], seed=27), 
                        param_grid = param_test1, 
                        scoring='roc_auc',
                        n_jobs=4,
                        iid=False, 
                        cv=5)

gsearch1.fit(train_X,train_y)
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 3, 'min_child_weight': 3}, 0.748077347456692)

In [41]:
xgb2 = XGBClassifier(
    learning_rate=0.1, n_estimators=1000,
    gamma=0, subsample=0.8, colsample_bytree=0.8,
    objective= 'binary:logistic', nthread=4, scale_pos_weight=counts[0]/counts[1], seed=27,
    max_depth = 3,
    min_child_weight = 3
)

xgb2.fit(train_X, train_y)

auc = roc_auc_score(train_y, xgb2.predict(train_X))
print("Performance sur le train : ", auc)

auc = roc_auc_score(test_y, xgb2.predict(test_X))
print("Performance sur le test : ", auc)

Performance sur le train :  0.9883647798742139
Performance sur le test :  0.6163936372269705


In [42]:
param_test3 = { 
    'gamma':[i/10.0 for i in range(0,5)] 
}
gsearch3 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=3,
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,
                                                  objective='binary:logistic', nthread=4, scale_pos_weight=counts[0]/counts[1], seed=27), 
                        param_grid = param_test3, 
                        scoring='roc_auc',
                        n_jobs=4,
                        iid=False, 
                        cv=5)

gsearch3.fit(train_X,train_y)
gsearch3.best_params_, gsearch3.best_score_

({'gamma': 0.4}, 0.7590554155102318)

In [43]:
xgb3 = XGBClassifier(
    learning_rate=0.1, 
    n_estimators=1000,
    gamma=0.4,
    subsample=0.8, colsample_bytree=0.8,
    objective='binary:logistic', nthread=4, scale_pos_weight=counts[0]/counts[1], seed=27,
    max_depth=3,
    min_child_weight=3
)

xgb3.fit(train_X, train_y)

auc = roc_auc_score(train_y, xgb3.predict(train_X))
print("Performance sur le train : ", auc)

auc = roc_auc_score(test_y, xgb3.predict(test_X))
print("Performance sur le test : ", auc)

Performance sur le train :  0.9765723270440252
Performance sur le test :  0.625534188034188


In [44]:
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=1000, 
                                                  max_depth=3,
                                                  min_child_weight=3,
                                                  gamma=0.4, 
                                                  subsample=0.8, colsample_bytree=0.8,
                                                  objective='binary:logistic', nthread=4, scale_pos_weight=counts[0]/counts[1], seed=27), 
                        param_grid = param_test4, 
                        scoring='roc_auc',
                        n_jobs=4,
                        iid=False, 
                        cv=5)

gsearch4.fit(train_X, train_y)
gsearch4.best_params_, gsearch4.best_score_


({'colsample_bytree': 0.8, 'subsample': 0.8}, 0.7590554155102318)

In [45]:
xgb5 = XGBClassifier(
    learning_rate=0.1, 
    n_estimators=1000,
    gamma=0.4,
    subsample=0.8,
    colsample_bytree=0.8,
    max_depth=3,
    min_child_weight=3,
    objective='binary:logistic', nthread=4, scale_pos_weight=counts[0]/counts[1], seed=27
)

xgb5.fit(train_X, train_y)

auc = roc_auc_score(train_y, xgb5.predict(train_X))
print("Performance sur le train : ", auc)

auc = roc_auc_score(test_y, xgb5.predict(test_X))
print("Performance sur le test : ", auc)

Performance sur le train :  0.9765723270440252
Performance sur le test :  0.625534188034188


In [46]:
param_test6 = {
 'learning_rate':[i/1000.0 for i in range(5,20,2)]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=1000, 
                                                  gamma=0.4,
                                                  subsample=0.8,
                                                  colsample_bytree=0.8,
                                                  max_depth=3,
                                                  min_child_weight=3,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=counts[0]/counts[1], seed=27), 
                        param_grid = param_test6, 
                        scoring='roc_auc',
                        n_jobs=4,
                        iid=False, 
                        cv=5)

gsearch6.fit(train_X, train_y)
gsearch6.best_params_, gsearch6.best_score_


({'learning_rate': 0.009}, 0.7722370755141361)

In [47]:
xgb6 = XGBClassifier(
    learning_rate=0.009,
    n_estimators=1000,
    max_depth=3,
    min_child_weight=3,
    gamma=0.4,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.005,
    objective= 'binary:logistic',
    nthread=4,
    scale_pos_weight=counts[0]/counts[1],
    seed=27
)

xgb6.fit(train_X, train_y)

auc = roc_auc_score(train_y, xgb6.predict(train_X))
print("Performance sur le train : ", auc)

auc = roc_auc_score(test_y, xgb6.predict(test_X))
print("Performance sur le test : ", auc)


Performance sur le train :  0.8711278391501287
Performance sur le test :  0.6692188983855651


In [48]:
tn, fp, fn, tp = confusion_matrix(train_y, xgb6.predict(train_X)).ravel()
# Error rate : 
err_rate = (fp + fn) / (tp + tn + fn + fp)
print("Error rate  : ", err_rate)
# Accuracy : 
acc_ = (tp + tn) / (tp + tn + fn + fp)
print("Accuracy  : ", acc_)
# Sensitivity : 
sens_ = tp / (tp + fn)
print("Sensitivity  : ", sens_)
# Specificity 
sp_ = tn / (tn + fp)
print("Specificity  : ", sens_)
# False positive rate (FPR)
FPR = fp / (tn + fp)
print("False positive rate  : ", FPR)

Error rate  :  0.16958197450340942
Accuracy  :  0.8304180254965906
Sensitivity  :  0.8251572327044026
Specificity  :  0.8251572327044026
False positive rate  :  0.08290155440414508


In [49]:
tn, fp, fn, tp = confusion_matrix(train_y, xgb6.predict(train_X)).ravel()
# Error rate : 
err_rate = (fp + fn) / (tp + tn + fn + fp)
print("Error rate  on train set : ", err_rate)
# Accuracy : 
acc_ = (tp + tn) / (tp + tn + fn + fp)
print("Accuracy  on train set  : ", acc_)

tn, fp, fn, tp = confusion_matrix(test_y, xgb6.predict(test_X)).ravel()
# Error rate : 
err_rate = (fp + fn) / (tp + tn + fn + fp)
print("Error rate  on test set : ", err_rate)
# Accuracy : 
acc_ = (tp + tn) / (tp + tn + fn + fp)
print("Accuracy  on test set  : ", acc_)

Error rate  on train set :  0.16958197450340942
Accuracy  on train set  :  0.8304180254965906
Error rate  on test set :  0.21955555555555556
Accuracy  on test set  :  0.7804444444444445
